In [10]:
# 导入pytorch这个包
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
torch.set_printoptions(precision=8)

In [11]:
class Detect(nn.Module):
    stride = None  # strides computed during build
    export = False  # onnx export

    def __init__(self, nc=80, anchors=(), ch=()):  # detection layer
        super(Detect, self).__init__()
        self.nc = nc  # number of classes 7
        self.no = nc + 5  # number of outputs per anchor 12
        self.nl = len(anchors)  # number of detection layers 3
        self.na = len(anchors[0]) // 2  # number of anchors 3
        self.grid = [torch.zeros(1)] * self.nl  # init grid
        a = torch.tensor(anchors).float().view(self.nl, -1, 2)
        self.register_buffer('anchors', a)  # shape(nl,na,2)
        self.register_buffer('anchor_grid', a.clone().view(self.nl, 1, -1, 1, 1, 2))  # shape(nl,1,na,1,1,2)
        self.m = nn.ModuleList(nn.Conv2d(x, self.no * self.na, 1) for x in ch)  # output conv

    def forward(self, x):
        # x = x.copy()  # for profiling
        z = []  # inference output
        self.training |= self.export
        for i in range(self.nl):
            x[i] = self.m[i](x[i])  # conv
            bs, _, ny, nx = x[i].shape  # x(bs,255,20,20) to x(bs,3,20,20,85)
            x[i] = x[i].view(bs, self.na, self.no, ny, nx).permute(0, 1, 3, 4, 2).contiguous()

            if not self.training:  # inference
                if self.grid[i].shape[2:4] != x[i].shape[2:4]:
                    self.grid[i] = self._make_grid(nx, ny).to(x[i].device)

                y = x[i].sigmoid()
                y[..., 0:2] = (y[..., 0:2] * 2. - 0.5 + self.grid[i].to(x[i].device)) * self.stride[i]  # xy
                y[..., 2:4] = (y[..., 2:4] * 2) ** 2 * self.anchor_grid[i]  # wh
                z.append(y.view(bs, -1, self.no))

        return x if self.training else (torch.cat(z, 1), x)

    @staticmethod
    def _make_grid(nx=20, ny=20):
        yv, xv = torch.meshgrid([torch.arange(ny), torch.arange(nx)])
        return torch.stack((xv, yv), 2).view((1, 1, ny, nx, 2)).float()

class MyDetect(nn.Module):
    def __init__(self, nc=80, anchors=(), ch=()):  # detection layer
        super(MyDetect, self).__init__()
        self.nc = nc  # number of classes 7
        self.no = nc + 5  # number of outputs per anchor 12
        self.nl = len(anchors)  # number of detection layers 3
        self.na = len(anchors[0]) // 2  # number of anchors 3
        #self.conv_out_1 = nn.Conv2d(ch[0], self.no * self.na, 1)
        #ones=torch.Tensor(np.ones([self.no * self.na, ch[0], 1, 1]))
        '''
        self.conv_out_1 = nn.Conv2d(ch[0], self.no * self.na, 1, 1, bias=True)
        self.conv_out_1.weight = torch.nn.Parameter(torch.Tensor(np.ones([self.no * self.na, ch[0], 1, 1])))
        self.conv_out_2 = nn.Conv2d(ch[1], self.no * self.na, 1, 1, bias=True)
        self.conv_out_2.weight = torch.nn.Parameter(torch.Tensor(np.ones([self.no * self.na, ch[1], 1, 1])))
        self.conv_out_3 = nn.Conv2d(512, 36, kernel_size=1, stride=1, bias=True)
        self.conv_out_3.weight = torch.nn.Parameter(torch.Tensor(np.ones([36, 512, 1, 1])))
        self.conv_out_3.bias = torch.nn.Parameter(torch.Tensor(np.zeros([36])))
        self.m = nn.ModuleList([self.conv_out_1, self.conv_out_2, self.conv_out_3])  # output conv
        '''
        self.m = nn.ModuleList(nn.Conv2d(x, self.no * self.na, 1) for x in ch)
        for i, x in enumerate(ch):
            self.m[i].weight = torch.nn.Parameter(torch.Tensor(np.ones([self.no * self.na, ch[i], 1, 1])))
            self.m[i].bias = torch.nn.Parameter(torch.Tensor(np.zeros([self.no * self.na])))

    def forward(self, x):
        for i in range(self.nl):
            x[i] = self.m[i](x[i])  # conv
        return x
    def my_forward(self, x):
        y = self.conv_out_3(x)  # conv
        return y    

class Concat(nn.Module):
    # Concatenate a list of tensors along dimension
    def __init__(self, dimension=1):
        super(Concat, self).__init__()
        self.d = dimension

    def forward(self, x):
        return torch.cat(x, self.d)

class SPP(nn.Module):
    # Spatial pyramid pooling layer used in YOLOv3-SPP
    def __init__(self, c1, c2, k=(5, 9, 13)):
        super(SPP, self).__init__()
        c_ = c1 // 2  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c_ * (len(k) + 1), c2, 1, 1)
        self.m = nn.ModuleList([nn.MaxPool2d(kernel_size=x, stride=1, padding=x // 2) \
                                for x in k])

    def forward(self, x):
        x = self.cv1(x)
        return self.cv2(torch.cat([x] + [m(x) for m in self.m], 1))

def autopad(k, p=None):  # kernel, padding
    # Pad to 'same'
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
    return p

class Conv(nn.Module):
    # Standard convolution
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, act=True):  # ch_in, ch_out, kernel, stride, padding, groups
        super(Conv, self).__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p), groups=g, bias=False)
        ones=torch.Tensor(np.ones([c2,c1,k,k])) 
        # 先创建一个自定义权值的Tensor，这里为了方便将所有权值设为1
        self.conv.weight=torch.nn.Parameter(ones) 
        # 把Tensor的值作为权值赋值给Conv层，这里需要先转为torch.nn.Parameter类型，否则将报错
        print(self.conv.weight.size())
        self.bn = nn.BatchNorm2d(c2)
        ##self.act = nn.Hardswish() if act else nn.Identity()
        self.act = nn.LeakyReLU(0.1, inplace=True) if act else nn.Identity()

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))
        #return self.conv(x)

    def fuseforward(self, x):
        return self.act(self.conv(x))
class Conv_(nn.Module):
    # Standard convolution
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, act=True):  # ch_in, ch_out, kernel, stride, padding, groups
        super(Conv_, self).__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, padding=autopad(k, p), groups=g, bias=False)
        ones=torch.Tensor(np.ones([c2,c1,k,k])) 
        # 先创建一个自定义权值的Tensor，这里为了方便将所有权值设为1
        self.conv.weight=torch.nn.Parameter(ones) 
        # 把Tensor的值作为权值赋值给Conv层，这里需要先转为torch.nn.Parameter类型，否则将报错
        print(self.conv.weight.size())
        self.bn = nn.BatchNorm2d(c2)
        ##self.act = nn.Hardswish() if act else nn.Identity()
        self.act = nn.LeakyReLU(0.1, inplace=True) if act else nn.Identity()

    def forward(self, x):
        return self.conv(x)
        return self.act(self.bn(self.conv(x)))
        #return self.conv(x)

    def fuseforward(self, x):
        return self.act(self.conv(x))

class Focus(nn.Module):
    # Focus wh information into c-space
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, act=True):  # ch_in, ch_out, kernel, stride, padding, groups
        super(Focus, self).__init__()
        self.conv = Conv(c1 * 4, c2, k, s, p, g, act)

    def forward(self, x):  # x(b,c,w,h) -> y(b,4c,w/2,h/2)
        return self.conv(torch.cat([x[..., ::2, ::2], x[..., 1::2, ::2], \
                                    x[..., ::2, 1::2], x[..., 1::2, 1::2]], 1))
class Bottleneck(nn.Module):
    # Standard bottleneck
    def __init__(self, c1, c2, shortcut=True, g=1, e=0.5):  # ch_in, ch_out, shortcut, groups, expansion
        super(Bottleneck, self).__init__()
        c_ = int(c2 * e)  # hidden channels
        print("Into Bottlneck")
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = Conv(c_, c2, 3, 1, g=g)
        self.add = shortcut and c1 == c2
        print("Out Bottleneck")

    def forward(self, x):
        zz = x + self.cv2(self.cv1(x)) if self.add else self.cv2(self.cv1(x))
        return zz


class BottleneckCSP(nn.Module):
    # CSP Bottleneck https://github.com/WongKinYiu/CrossStagePartialNetworks
    def __init__(self, c1, c2, n=1, shortcut=True, g=1, e=0.5):  # ch_in, ch_out, number, shortcut, groups, expansion
        super(BottleneckCSP, self).__init__()
        c_ = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, c_, 1, 1)
        self.cv2 = nn.Conv2d(c1, c_, 1, 1, bias=False)
        ones=torch.Tensor(np.ones([c_,c1,1,1])) 
        # 先创建一个自定义权值的Tensor，这里为了方便将所有权值设为1
        self.cv2.weight=torch.nn.Parameter(ones) 
        # 把Tensor的值作为权值赋值给Conv层，这里需要先转为torch.nn.Parameter类型，否则将报错
        print(self.cv2.weight.size())
        self.cv3 = nn.Conv2d(c_, c_, 1, 1, bias=False)
        ones=torch.Tensor(np.ones([c_,c_,1,1]))
        # 先创建一个自定义权值的Tensor，这里为了方便将所有权值设为1
        self.cv3.weight=torch.nn.Parameter(ones) 
        # 把Tensor的值作为权值赋值给Conv层，这里需要先转为torch.nn.Parameter类型，否则将报错
        print(self.cv3.weight.size())
        self.cv4 = Conv(2 * c_, c2, 1, 1)
        self.bn = nn.BatchNorm2d(2 * c_)  # applied to cat(cv2, cv3)
        self.act = nn.LeakyReLU(0.1, inplace=True)
        self.m = nn.Sequential(*[Bottleneck(c_, c_, shortcut, g, e=1.0) for _ in range(n)])

    def forward(self, x):
        y1 = self.cv3(self.m(self.cv1(x)))
        y2 = self.cv2(x)
        return self.cv4(self.act(self.bn(torch.cat((y1, y2), dim=1))))


In [12]:
class YOLOV5S(nn.Module):
    def __init__(self):
        super(YOLOV5S, self).__init__()
    def forward(self, x):
        print("in yolov5s forward")
        
        #net_ = Focus(c1=3, c2=32, k=3)
        #y = net_(x)
        net_ = Conv(c1=3, c2=32, k=3, s=1)
        y = net_(x)
        #################################
        conv1 = Conv(c1=32, c2=64, k=3, s=2)
        out = conv1(y)
        ################################
        print("Into BottleneckCSP")
        BottleneckCSP_0 = BottleneckCSP(c1=64, c2=64)
        out_botcsp = BottleneckCSP_0(out)
        print("Leav BottleneckCSP")
        conv_d_2 = Conv(c1=64, c2=128, k=3, s=2)
        out_conv_d_2 = conv_d_2(out_botcsp)
        BottleneckCSP_1 = BottleneckCSP(c1=128, c2=128, n=3)
        out_botcsp1 = BottleneckCSP_1(out_conv_d_2)
        conv_d_3 = Conv(c1=128, c2=256, k=3, s=2)
        out_conv_d_3 = conv_d_3(out_botcsp1)
        BottleneckCSP_２ = BottleneckCSP(c1=256, c2=256, n=3)
        out_botcsp2 = BottleneckCSP_２(out_conv_d_3)
        conv_d_4 = Conv(c1=256, c2=512, k=3, s=2)
        out_conv_d_4 = conv_d_4(out_botcsp2)
        spp = SPP(512,512)
        spp_out = spp(out_conv_d_4)
        
        BottleneckCSP_3 = BottleneckCSP(c1=512, c2=512, n=1, shortcut=False)
        out_botcsp3 = BottleneckCSP_3(spp_out)
        conv_5 = Conv(c1=512, c2=256)
        out_conv_5 = conv_5(out_botcsp3)
        upsample_1 = nn.modules.upsampling.Upsample(scale_factor=2,mode='nearest')
        out_upsample_1 = upsample_1(out_conv_5)
        concat_1 = Concat(dimension=1)
        out_concat_1 = concat_1([out_upsample_1, out_botcsp2])
        
        BottleneckCSP_4 = BottleneckCSP(c1=512, c2=256, n=1, shortcut=False)
        out_botcsp4 = BottleneckCSP_4(out_concat_1)        
        conv_6 = Conv(c1=256, c2=128)
        out_conv_6 = conv_6(out_botcsp4)
        upsample_2 = nn.modules.upsampling.Upsample(scale_factor=2,mode='nearest')
        out_upsample_2 = upsample_2(out_conv_6)
        concat_2 = Concat(dimension=1)
        out_concat_2 = concat_2([out_upsample_2, out_botcsp1])
        
        BottleneckCSP_5 = BottleneckCSP(c1=256, c2=128, n=1, shortcut=False)
        out_botcsp5 = BottleneckCSP_5(out_concat_2)
        conv_d_5 = Conv(c1=128, c2=128, k=3, s=2)
        out_conv_d_5 = conv_d_5(out_botcsp5)
        concat_3 = Concat(dimension=1)
        out_concat_3 = concat_3([out_conv_d_5, out_conv_6])
        
        BottleneckCSP_6 = BottleneckCSP(c1=256, c2=256, n=1, shortcut=False)
        out_botcsp6 = BottleneckCSP_6(out_concat_3)
        conv_d_6 = Conv(c1=256, c2=256, k=3, s=2)
        out_conv_d_6 = conv_d_6(out_botcsp6)
        concat_4 = Concat(dimension=1)
        out_concat_4 = concat_4([out_conv_d_6, out_conv_5])
        
        BottleneckCSP_7 = BottleneckCSP(c1=512, c2=512, n=1, shortcut=False)
        out_botcsp7 = BottleneckCSP_7(out_concat_4)
        
        myDetect_all = MyDetect(nc=7, anchors=[[10, 13, 16, 30, 33, 23], [30, 61, 62, 45, 59, 119], [116, 90, 156, 198, 373, 326]], ch=[128, 256, 512])
        detect_out = myDetect_all([out_botcsp5, out_botcsp6, out_botcsp7])
                                
        return detect_out

In [13]:
import cv2
import numpy as np
img_file = '/data/github_repos/yolov3-tiny-fit-ncs/ncs/standard_size_416_persons.jpg'
cv2.__version__
img = cv2.imread(img_file) ###BGR
resized_img = cv2.resize(img, (208,208))
resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB) # cv2默认为bgr顺序
img_ = np.transpose(resized_img, (2, 0, 1))
#print(img_.shape)
#print(img_)
normerlized = img_/255.0
input_t = torchvision.transforms.functional.to_tensor(resized_img)
input_t = input_t.unsqueeze(0)
print(input_t)
#print(normerlized)
net = YOLOV5S()
output = net(input_t)
for o in output:
    print(o.size())
    print(o)

tensor([[[[0.04705882, 0.12941177, 0.14509805,  ..., 0.11372549,
           0.11764706, 0.10980392],
          [0.13333334, 0.69803923, 0.74509805,  ..., 0.01176471,
           0.03921569, 0.14117648],
          [0.23137255, 0.58039218, 0.42745098,  ..., 0.05882353,
           0.01176471, 0.01176471],
          ...,
          [0.21960784, 0.21960784, 0.23137255,  ..., 0.41568628,
           0.41176471, 0.40000001],
          [0.21568628, 0.22352941, 0.23137255,  ..., 0.40784314,
           0.40000001, 0.40000001],
          [0.21176471, 0.21960784, 0.23137255,  ..., 0.40784314,
           0.40000001, 0.41568628]],

         [[0.09019608, 0.14117648, 0.16470589,  ..., 0.30980393,
           0.33333334, 0.24705882],
          [0.16862746, 0.71372551, 0.76862746,  ..., 0.05882353,
           0.15686275, 0.29019609],
          [0.27450982, 0.59215689, 0.44705883,  ..., 0.06274510,
           0.03921569, 0.05490196],
          ...,
          [0.23529412, 0.23529412, 0.24705882,  ..., 0.4313

tensor([[[[-6.67718887, -6.67885113, -6.67887688,  ..., -6.67887688,
           -6.67887688, -6.67773867],
          [-6.67887688, -6.68138266, -6.68139219,  ..., -6.68139219,
           -6.68139219, -6.67969179],
          [-6.67887688, -6.68139219, -6.68139219,  ..., -6.68139219,
           -6.68139219, -6.67969179],
          ...,
          [-6.67887688, -6.68139219, -6.68110895,  ..., -6.68139219,
           -6.68139219, -6.67969179],
          [-6.67887688, -6.68139219, -6.68083334,  ..., -6.68139219,
           -6.68139219, -6.67969179],
          [-6.67773867, -6.67969179, -6.67968702,  ..., -6.67969179,
           -6.67969179, -6.67858410]],

         [[-6.67718887, -6.67885113, -6.67887688,  ..., -6.67887688,
           -6.67887688, -6.67773867],
          [-6.67887688, -6.68138266, -6.68139219,  ..., -6.68139219,
           -6.68139219, -6.67969179],
          [-6.67887688, -6.68139219, -6.68139219,  ..., -6.68139219,
           -6.68139219, -6.67969179],
          ...,
     

In [ ]:
torch.load()